<a href="https://colab.research.google.com/github/dgo2dance/qwen-tune/blob/main/LLaMA-Factory-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check GPU configuration
!nvidia-smi

Sun Dec 10 02:30:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#STEP 0 確認有使用GPU
##記事本必須在GPU模式下運行，可以執行上方的nvidia-smi查看GPU是否有GPU。若没有，點擊左上角的 編輯>筆記本設定，把硬體加速器改成GPU。
##如記事本有出問題可在以下儲存庫回報
> https://github.com/ADT109119/LLaMA-Factory-Colab

In [ ]:
#@title STEP 1 複製儲存庫並安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##複製儲存庫並安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib

!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd ./LLaMA-Factory
!pip install -r requirements.txt
!pip install bitsandbytes>=0.39.0

#用比較殘暴的方式開啟 Gradio 的分享連結
train_web_py_file_path = "./src/train_web.py"
try:
  with open(train_web_py_file_path, 'r') as file:
    file_content = file.read()
  modified_content = file_content.replace("share=False", "share=True")
  with open(train_web_py_file_path, 'w') as file:
    file.write(modified_content)
except Exception as e:
    print(f'ERROR: {str(e)}')

In [6]:
! pip install cohere  openai tiktoken
! pip install transformers_stream_generator

  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.4-py3-none-any.whl size=12316 sha256=ca6654971187d829ea53e3a81bb11e9f491f0ace705a2b79d711ba48e448265f
  Stored in directory: /root/.cache/pip/wheels/47/1d/3c/92d88493ed40c0d9be60a391eb76c9a56e9f9b7542cb789401
Successfully built transformers_stream_generator


In [7]:
#@title STEP 2 開啟WebUI
#@markdown #STEP 2
#@markdown ##開啟WebUI
#@markdown ##Run the WebUI

!CUDA_VISIBLE_DEVICES=0 python src/train_web.py

2024-01-23 06:04:02.889127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 06:04:02.889203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 06:04:02.891270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 06:04:05.316013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://8b68a7298f5eaa4708.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces

In [8]:
! pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.26.1
aiofiles                         23.2.1
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
b

#STEP 3 上傳輸出的檔案到 HuggingFace


In [ ]:
#@title STEP 3.1 安裝HuggingFace API
#@markdown #STEP 3.1
#@markdown ##為了上傳模型至HuggingFace，需要先安裝HF的函式庫
#@markdown ##In order to upload the model to HuggingFace, we have to install the HF library first.
!pip install huggingface_hub

In [ ]:
#@title STEP 3.2 使用API Token登入
#@markdown #STEP 3.2
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title STEP 3.3 上傳模型
#@markdown #STEP 3.3
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login
from huggingface_hub import HfApi
api = HfApi()

model_id = "username/fine-tune-model" #@param {type:"string"}
export_folder_path = "/content/LLaMA-Factory/export" #@param {type:"string"}
api.create_repo(model_id, private=True, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=export_folder_path,
    repo_id=model_id,
)

#STEP 4 保存到Google Drive(自選)

In [ ]:
#@title STEP 4.1 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 4.2 複製檔案到Google Drive
Google_Drive_Folder = "/content/drive/MyDrive/LLaMA-Factory/" #@param {type:"string"}

!cp -rf /content/LLaMA-Factory/saves {Google_Drive_Folder}